In [ ]:
#importing libraries
import pandas as pd
import csv

In [ ]:
#reading the excel file
df = pd.read_csv(r'RAW DATA FILE.csv')
df["balance"].fillna(0,inplace = True)

In [ ]:
#column to be used for determining the status - "info"
row_count = sum(1 for row in df["info"])  # we calculate the number of entries to determine the loop

In [ ]:
stat = []
for i in range(0,row_count):
    val = (df["info"][i]).split(",")
    bal = df["balance"][i]
    
    #Payment successful 
    if (val[0] == "Autopay success DOB" or val[0] == "Person answered DOB") and "IM-Success" in val and bal != 0:
        stat.append("Payment Successful")
    
    #IM-Success with no Payment
    elif (val[0] == "Autopay success DOB" or val[0] == "Person answered DOB") and "IM-Success" in val and bal == 0:
        stat.append("IM-Success with no Payment")
        
    #Transfer to Office
    elif val[0] == "Transfer to office DOB" and val[-1] == "Trans":
        stat.append("Transfer to Office")
        
    #Dropped Call
    elif len(val) == 1 and val[0] == "Person answered":
        stat.append("Dropped Call")
        
    elif val[0] == "Person answered DOB" and (val[len(val)-1] == "MMenu" or val[len(val)-1] == "MoreOpts" or val[len(val)-1] == "OfficeHrs"):
        stat.append("Dropped Call")
        
    #Declined Payment
    elif ("IM-failure" in val) and ("IM-Success" not in val) and (val[0] == "Person answered DOB" or val[0] == "Autopay failure DOB"):
        stat.append("Declined Payment")
        
    #Stopped at DOB
    elif (len(val) == 1 and val[0] == "Person answered DOB") or ((len(val) > 1 and val[0] == "Person answered DOB") and all(p == "SOver" or p == "DOB" for p in val[1:])):
        stat.append("Stopped at DOB")
        
    #Phone Vaidation Failed
    elif (val[len(val)-1] == "phone") or (val[len(val)-2] == "phone" and (val[len(val)-1] == "Bye" or val[len(val)-1] == "accnt#")):
        stat.append("Phone Validation Failed")
        
    #Stopped at Zip
    elif val[0] == "Person answered DOB" and (val[-1] == "AuthZip" or (val[-2] == "AuthZip" and val[-1] == "Bye")) and "DOB" not in val:
        stat.append("Stopped at Zip")
        
    #Tried Another DOB/Zip
    elif val[0] == "Person answered DOB" and val[1] == "AuthZip" and (val[2] == "DOB" or val[2] == "SOver") and "IM-Success" not in val:
        stat.append("Tried Another DOB/Zip")
        
    #Validated - Dropped during Payment
    elif 'PayFull' in val and 'IM-Success' not in val:
        stat.append("Validated - Dropped during Payment")
        
    #Validated then Dropped
    elif val[0] == "Person answered DOB" and val[len(val)-1] == "MMenu":
        if len(val) == 3 and val[1] == "AuthZip":
            stat.append("Validated then Dropped")
        else:
            stat.append("error")
            
    else:
        stat.append("NEW VAL")

In [ ]:
df["Status"] = stat
df.to_csv(r"SAME RAW DATA FILE WILL BE CHANGED.csv")